<a href="https://colab.research.google.com/github/ridaIndah/Proyek-Mandiri/blob/main/train4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **cek Cuda (True or False)**

In [ ]:
import torch
torch.cuda.is_available() #should be true

True

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
!nvidia-smi

Sat Jun  5 10:19:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
torch.backends.cudnn.is_available()

True

In [ ]:
#install pymagnitude for word embedding
pip install pymagnitude

     |████████████████████████████████| 5.4MB 6.5MB/s 
  Created wheel for pymagnitude: filename=pymagnitude-0.1.143-cp37-cp37m-linux_x86_64.whl size=360640573 sha256=5f2993f2118f72e468b2691aa6e937857be70e34619eec8a159adb4233838b6c
  Stored in directory: /root/.cache/pip/wheels/bb/b2/16/f88eb50800667ffe23d509c7e2157923a08d10ed6d6410124f
Successfully built pymagnitude


In [ ]:
import os
import random
import logging
import argparse

import numpy as np
import torch
from torch import nn
from torch import optim
from torch.nn.init import xavier_uniform_


In [ ]:
from pymagnitude import Magnitude

# Prepare data

In [ ]:

def load_pickle(fname):
    with open(fname, "rb") as f:
        return pickle.load(f)

In [ ]:
#for preproses data DD
import re
import csv
import random
from collections import Counter, defaultdict
import nltk
nltk.download('stopwords')
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import spacy
from spacy.lang.en.stop_words import STOP_WORDS as spacy_stopwords


DD_label_to_emotions = {0: "neutral", 1: "anger", 2: "disgust", 3: "fear", 4: "happiness", 5: "sadness", 6: "surprise"}
nltk_stopwords = stopwords.words('english')
# spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS # older version of spacy
stopwords = set(nltk_stopwords).union(spacy_stopwords)
porter = PorterStemmer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#Define class Vocab for building vocabulary
class Vocab(object):
    def __init__(self, examples, min_freq, max_vocab_size):
        """
            examples: a list of examples, each example is a list of (utter, speaker, emotion, mask), followed by an emotion label
            min_freq: the min frequency of word in the vocabulary
            max_vocab_size: max vocabulary size
            return: a Vocab object
        """
        self.min_freq = min_freq
        self.max_vocab_size = max_vocab_size
        
        self.word2id = {"<pad>": 0, "<unk>": 1}
        self.id2word = {0: "<pad>", 1: "<unk>"}
        self.speaker2id = {}
        self.emotion2id = {}

        self.word_freq_dist = Counter()
        self.speaker_freq_dist = Counter()
        self.emotion_freq_dist = Counter()
        
        utterance_lengths = []
        conversation_lengths = []

        for ex in examples:
            conv_length = sum([utter[-1] for utter in ex])
            conversation_lengths.append(conv_length)
            for utter, speaker, emotion, mask in ex:
                if mask == 1:
                    self.word_freq_dist.update(utter)
                    self.speaker_freq_dist.update([speaker])
                    self.emotion_freq_dist.update([emotion])
                    utterance_lengths.append(len(utter))

        # filter by min_freq
        words = [(w,cnt) for w, cnt in self.word_freq_dist.items() if cnt >= min_freq]
        words = sorted(words, key=lambda x: x[1], reverse=True)

        # cap vocab size
        words = words[:max_vocab_size]

        # build word2id and id2word
        for idx, (w, cnt) in enumerate(words):
            self.word2id[w] = idx+2
            self.id2word[idx+2] = w

        # build speaker to id
        speakers = sorted(self.speaker_freq_dist.items(), key=lambda x: x[1], reverse=True)
        for idx, (speaker, cnt) in enumerate(speakers):
            self.speaker2id[speaker] = idx

        # build emotion to id
        emotions = sorted(self.emotion_freq_dist.items(), key=lambda x: x[1], reverse=True)
        for idx, (emotion, cnt) in enumerate(emotions):
            self.emotion2id[emotion] = idx
        
        self.max_conversation_length = max(conversation_lengths)
        self.max_sequence_length = max(utterance_lengths)
        self.num_utterances = len(utterance_lengths)
        

def convert_examples_to_ids(examples, vocab):
    """
        examples: a list of examples, each example is a list of (utter, speaker, emotion, mask), followed by an emotion label
        vocab: a Vocab object
        max_sequence_length: max sequence length
        
        return: examples containing word ids
    """
    examples_ids = []
    for ex in examples:
        ex_ids = []
        for utter, speaker, emotion, mask in ex:
            utter_ids = [vocab.word2id[w] if w in vocab.word2id else vocab.word2id["<unk>"] for w in utter]
            utter_ids = utter_ids + (vocab.max_sequence_length - len(utter_ids)) * [vocab.word2id["<pad>"]]
            if speaker in vocab.speaker2id:
                speaker_id = vocab.speaker2id[speaker]
            else:
                speaker_id = len(vocab.speaker2id) # val or test set, use a new speaker id
            emotion_id = vocab.emotion2id[emotion]
            ex_ids.append((utter_ids, speaker_id, emotion_id, mask))
        examples_ids.append(ex_ids)
    return examples_ids

def create_one_batch(examples):
    """
        examples: a batch of examples having the same number of turns and seq_len, each example is a list of (utter, speaker, label, mask)
        
        return: batch_x, batch_y, where batch_x is a tuple of token ids and speaker ids
    """
    tokens = []
    speakers = []
    labels = []
    masks = []
    for ex in examples:
        ex_tokens, ex_speakers, ex_labels, ex_masks = list(zip(*ex))
        tokens.append(ex_tokens)
        speakers.append(ex_speakers)
        labels.append(ex_labels)
        masks.append(ex_masks)
    
    return (tokens, speakers), labels, masks


def create_batches(examples, batch_size, train=True):
    batch_data = []
    if train == True:
        random.shuffle(examples)
    
    batch_ids = list(range(0, len(examples), batch_size)) + [len(examples)]
    for s, e in zip(batch_ids[:-1], batch_ids[1:]):
        batch_examples = examples[s:e]
        batch_x, batch_y, batch_mask = create_one_batch(batch_examples)
        batch_data.append((batch_x, batch_y, batch_mask))
    return batch_data


def create_balanced_batches(examples, batch_size, train=True):
    batch_data = []
    if train == True:
        random.shuffle(examples)
    
    num_valid_utterances = []
    for ex in examples:
        num_valid_utterances.append(sum([mask for utter, speaker, label, mask in ex]))
    
    batch_ids = [0]
    for i in range(len(examples)):
        if len(num_valid_utterances[batch_ids[-1]:i]) >= batch_size:
            batch_ids.append(i)
    
    for s, e in zip(batch_ids[:-1], batch_ids[1:]):
        batch_examples = examples[s:e]
        batch_x, batch_y, batch_mask = create_one_batch(batch_examples)
        batch_data.append((batch_x, batch_y, batch_mask))
    return batch_data


def merge_splits(train, val):
    if len(train[0]) > len(val[0]):
        num_additional_utterances = len(train[0]) - len(val[0])
        for ex in val:
            if ex[-1][0] == ['this', 'is', 'a', 'dummy', 'sentence']:
                dummy_ex = ex[-1]
                break
        new_val = []
        for ex in val:
            new_val.append(ex + num_additional_utterances*[dummy_ex])
        return train + new_val
    elif len(train[0]) < len(val[0]):
        num_additional_utterances = len(val[0]) - len(train[0])
        for ex in train:
            if ex[-1][0] == ['this', 'is', 'a', 'dummy', 'sentence']:
                dummy_ex = ex[-1]
                break
        new_train = []
        for ex in train:
            new_train.append(ex + num_additional_utterances*[dummy_ex])
        return new_train + val
    else:
        return train + val

# conceptnet
def get_vocab_embedding(vocab, vectors, embedding_size):
    pretrained_word_embedding = np.zeros((len(vocab.word2id), embedding_size))
    for w, i in vocab.word2id.items():
        pretrained_word_embedding[i] = vectors.query(w)
    return pretrained_word_embedding


def get_emotion_intensity(NRC, word):
    if word not in NRC:
        word = porter.stem(word)
        if word not in NRC:
            return 0.5
    v, a, d = NRC[word]
    a = a/2
    return (np.linalg.norm(np.array([v, a]) - np.array([0.5, 0])) - 0.06467)/0.607468


# edge matrix construction
def filter_conceptnet(conceptnet, vocab):
    filtered_conceptnet = {}
    for k in conceptnet:
        if k in vocab.word2id and k not in stopwords:
            filtered_conceptnet[k] = set()
            for c,w in conceptnet[k]:
                if c in vocab.word2id and c not in stopwords and w>=1:
                    filtered_conceptnet[k].add((c,w))
    return filtered_conceptnet


# remove cases where the same concept has multiple weights
def remove_KB_duplicates(conceptnet):
    filtered_conceptnet = {}
    for k in conceptnet:
        filtered_conceptnet[k] = set()
        concepts = set()
        filtered_concepts = sorted(conceptnet[k], key=lambda x: x[1], reverse=True)
        for c,w in filtered_concepts:
            if c not in concepts:
                filtered_conceptnet[k].add((c, w))
                concepts.add(c)
    return filtered_conceptnet

# Prepare tools

In [ ]:
from collections import Counter
import numpy as np


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def counter_to_distribution(counter):
    distribution = {}
    total = sum([v for k, v in counter.items()])
    for k, v in counter.items():
        distribution[k] = round(v/total, 4)
    return distribution


def label_distribution_transformer(examples):
    counter = Counter([ex[-1] for ex in examples])
    percentatges = []
    for i in range(len(counter)):
        percentatges.append(counter[i]/len(examples))
    return percentatges

# Prepare model transformer

embedding

In [ ]:
#embedding
print_dims = False
import torch
import torch.nn as nn
import math

class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.embedding = nn.Embedding(vocab, d_model)
        self.d_model = d_model
        
    def forward(self, x):
        """
            x: (batch_size, seq_len)
        """
        if print_dims:
            print("{0}: x: type: {1}, shape: {2}".format(self.__class__.__name__, x.type(), x.shape))
        return self.embedding(x) * math.sqrt(self.d_model)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # compute the positional encodings once in log space
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0.0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0.0, d_model, 2) * -(math.log(10000.0)/d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        """
            x: (batch_size, seq_len, d_model)
        """
        if print_dims:
            print("{0}: x: type: {1}, shape: {2}".format(self.__class__.__name__, x.type(), x.shape))
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

module

In [ ]:
#import module (inside attention)
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
print_dims = False

class LayerNorm(nn.Module):
    "Construct a layernorm module"
    def __init__(self, n_features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(n_features))
        self.b_2 = nn.Parameter(torch.zeros(n_features))
        self.eps = eps
        
    def forward(self, x):
        """
            x: (batch_size, seq_len, d_model)
        """
        if print_dims:
            print("{0}: x: type: {1}, shape: {2}".format(self.__class__.__name__, x.type(), x.shape))
        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, keepdim=True)
        return self.a_2 * (x - mean)/(std + self.eps) + self.b_2

class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    The norm is applied first as apposed to last
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, sublayer):
        """
            x: (batch_size, seq_len, d_model)
        """
        if print_dims:
            print("{0}: x: type: {1}, shape: {2}".format(self.__class__.__name__, x.type(), x.shape))
        "Apply residual connection to any sublayer with the same size"
        return x + self.dropout(sublayer(self.norm(x)))

class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionWiseFeedForward, self).__init__()
        self.w1 = nn.Linear(d_model, d_ff)
        self.w2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        """
            x: (batch_size, seq_len, d_model)
        """
        if print_dims:
            print("{0}: x: type: {1}, shape: {2}".format(self.__class__.__name__, x.type(), x.shape))
        return self.w2(self.dropout(F.relu(self.w1(x))))


def clones(module, N):
    """
    Produce N identical layers
    """
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])


attention

In [ ]:
# including graph attention (proposed in this project)

def attention(query, key, value, mask=None, dropout=None):
    """
    Compute scaled dot product self attention
        query: (batch_size, h, seq_len, d_k), seq_len can be either src_seq_len or tgt_seq_len
        key: (batch_size, h, seq_len, d_k), seq_len in key, value and mask are the same
        value: (batch_size, h, seq_len, d_k)
        mask: (batch_size, 1, 1, seq_len) or (batch_size, 1, tgt_seq_len, tgt_seq_len) (legacy)
    """
    if print_dims:
        print("{0}: query: type: {1}, shape: {2}".format("attention func", query.type(), query.shape))
        print("{0}: key: type: {1}, shape: {2}".format("attention func", key.type(), key.shape))
        print("{0}: value: type: {1}, shape: {2}".format("attention func", value.type(), value.shape))
        print("{0}: mask: type: {1}, shape: {2}".format("attention func", mask.type(), mask.shape))
    d_k = query.size(-1)

    # scores: (batch_size, h, seq_len, seq_len) for self_attn, (batch_size, h, tgt_seq_len, src_seq_len) for src_attn
    scores = torch.matmul(query, key.transpose(-2, -1)/math.sqrt(d_k)) 
    # print(query.shape, key.shape, mask.shape, scores.shape)

    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

class MultiHeadAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        print("Nilai H :", h)
        print("Nilai d_model :", d_model)
        super(MultiHeadAttention, self).__init__()
        assert d_model % h == 0
        self.d_k = d_model//h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        self.a = nn.Parameter(torch.Tensor([1]))
        self.b = nn.Parameter(torch.Tensor([1]))
        
    def forward(self, query, key, value, mask=None):
        """
            query: (batch_size, seq_len, d_model), seq_len can be either src_seq_len or tgt_seq_len
            key: (batch_size, seq_len, d_model), seq_len in key, value and mask are the same
            value: (batch_size, seq_len, d_model)
            mask: (batch_size, 1, seq_len) or (batch_size, tgt_seq_len, tgt_seq_len) (legacy)
        """
        if print_dims:
            print("{0}: query: type: {1}, shape: {2}".format(self.__class__.__name__, query.type(), query.shape))
            print("{0}: key: type: {1}, shape: {2}".format(self.__class__.__name__, key.type(), key.shape))
            print("{0}: value: type: {1}, shape: {2}".format(self.__class__.__name__, value.type(), value.shape))
            print("{0}: mask: type: {1}, shape: {2}".format(self.__class__.__name__, mask.type(), mask.shape))
        if mask is not None:
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all linear projections in batch from d_model to (h, d_k)
        query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1,2)
            for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout) # (batch_size, h, seq_len, d_k)
        if print_dims:
            print("{0}: x (after attention): type: {1}, shape: {2}".format(self.__class__.__name__, x.type(), x.shape))

        # 3) Concatenate and apply a final linear
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        x = self.linears[-1](x) # (batch_size, seq_len, d_model)
        if print_dims:
            print("{0}: x (after concatenation and linear): type: {1}, shape: {2}".format(self.__class__.__name__, x.type(), x.shape))
        return x


class GraphAttention(nn.Module):
    def __init__(self, vocab_size, embed_size, GAW=None, variant=0, concept_embed=None):
        super(GraphAttention, self).__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.variant = variant
        if concept_embed is None:
            self.concept_embed = nn.Embedding(vocab_size, embed_size)
        else:
            self.concept_embed = concept_embed

    def init_params(self, GAW=None, edge_matrix=None, affectiveness=None, concentration_factor=1):
        self.GAW = GAW
        edge_matrix_range = (edge_matrix.max(dim=1)[0] - edge_matrix.min(dim=0)[0]).unsqueeze(1)
        edge_matrix = edge_matrix/(edge_matrix_range + (edge_matrix_range==0).float()) # normalization
        self.edge_matrix = edge_matrix
        self.affectiveness = affectiveness
        self.concentration_factor = concentration_factor
        if self.GAW is not None:
            self._lambda = self.GAW
        else:
            self._lambda = nn.Parameter(torch.full((self.vocab_size,), 0.5))


    def get_hierarchical_sentence_representation(self, sent_embed):
        # sent_embed: (batch_size, seq_len, d_model)
        seq_len = sent_embed.shape[1]
        N = 3 # n-gram hierarchical pooling
        
        # max pooling for each ngram
        ngram_embeddings = [[] for i in range(N-1)] # one list for each n
        for n in range(1, N):
            for i in range(seq_len):
                ngram_embeddings[n-1].append(sent_embed[:,i:i+n+1,:].max(dim=1)[0])
        
        # mean pooling across ngram embeddings
        pooled_ngram_embeddings = [sent_embed.mean(dim=1)] # unigram
        for ngram_embedding in ngram_embeddings:
            ngram_embedding = torch.stack(ngram_embedding, dim=1).mean(dim=1)
            pooled_ngram_embeddings.append(ngram_embedding)

        sent_embed = torch.stack(pooled_ngram_embeddings, dim=1).mean(dim=1)
        return sent_embed

    def get_context_representation(self, src_embed, tgt_embed):
        # src_embed: (batch_size, context_length*seq_len, d_model)
        # tgt_embed: (batch_size, seq_len, d_model)
        seq_len = tgt_embed.shape[1]
        context_length = src_embed.shape[1]//seq_len
        sentence_representations = []
        for i in range(context_length):
            sentence_representations.append(self.get_hierarchical_sentence_representation(src_embed[:, i*seq_len:(i+1)*seq_len]))
        sentence_representations.append(self.get_hierarchical_sentence_representation(tgt_embed))
        context_representation = torch.stack(sentence_representations, dim=1).mean(dim=1) # (batch_size, d_model)
        return context_representation

    def forward(self, src, src_embed, tgt, tgt_embed):
        # src: (batch_size, context_length * seq_len)
        # src_embed: (batch_size, context_length*seq_len, d_model)
        # tgt: (batch_size, seq_len)
        # tgt_embed: (batch_size, seq_len, d_model)
        # embed: shared embedding layer: (vocab_size, d_model)

        # get context representation
        if self.variant == 1:
            # standard attention as in GAT
            src_len = src.shape[1]
            src = torch.cat([src, tgt], dim=1)
            src_embed = torch.cat([src_embed, tgt_embed], dim=1) # (batch_size, (context_length+1)*seq_len, d_model), self.concept_embed.weight: (vocab_size, d_model)
            concept_weights = (self.edge_matrix[src] > 0).float() * torch.matmul(src_embed, self.concept_embed.weight.transpose(0,1)) # (batch_size, (context_length+1)*seq_len, vocab_size)
            concept_embedding = torch.matmul(torch.softmax(concept_weights * self.concentration_factor, dim=2), self.concept_embed.weight)
            return concept_embedding[:, :src_len, :], concept_embedding[:, src_len:, :]
        if self.variant == 2:
            context_representation = self.get_context_representation(src_embed, tgt_embed) # (batch_size, d_model)
            # get concept embedding
            src_len = src.shape[1]
            src = torch.cat([src, tgt], dim=1)
            cosine_similarity = torch.abs(torch.cosine_similarity(context_representation.unsqueeze(1), \
                self.concept_embed.weight.unsqueeze(0), dim=2)) # (batch_size, vocab_size)
            cosine_similarity = cosine_similarity.to(device)
            relatedness = self.edge_matrix[src] * cosine_similarity.unsqueeze(1) # (batch_size, (context_length+1)*seq_len, vocab_size)
            concept_weights = self._lambda*relatedness + (1-self._lambda)*(self.edge_matrix[src] > 0).float()*self.affectiveness # (batch_size, (context_length+1)*seq_len, vocab_size)
            concept_embedding = torch.matmul(torch.softmax(concept_weights * self.concentration_factor, dim=2), self.concept_embed.weight) # (batch_size, (context_length+1)*seq_len, d_model)
            return concept_embedding[:, :src_len, :], concept_embedding[:, src_len:, :]
            print("cek cosine")
            cosine_similarity.is_cuda()

encoder

In [ ]:
#encoder
import torch
import torch.nn as nn
print_dims = False
class Encoder(nn.Module):
    "Encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        """
            x: (batch_size, src_seq_len, d_model)
            mask: (batch_size, 1, src_seq_len)
        """
        "Pass the input token ids and mask through each layer in turn"
        if print_dims:
            print("{0}: x: type: {1}, shape: {2}".format(self.__class__.__name__, x.type(), x.shape))
            print("{0}: mask: type: {1}, shape: {2}".format(self.__class__.__name__, mask.type(), mask.shape))
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)


class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward layers"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size
        
    def forward(self, x, mask):
        """norm -> self_attn -> dropout -> add -> 
        norm -> feed_forward -> dropout -> add"""
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

decoder

In [ ]:
#decoder
import torch
import torch.nn as nn
import torch.nn.functional as F
print_dims = False

class Decoder(nn.Module):
    "Generic N layer decoder with masking"
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, src_states, src_mask, tgt_mask):
        """
            x: (batch_size, tgt_seq_len, d_model)
            src_states: (batch_size, src_seq_len, d_model)
            src_mask: (batch_size, 1, src_seq_len)
            tgt_mask: (batch_size, tgt_seq_len, tgt_seq_len)
        """
        if print_dims:
            print("{0}: x: type: {1}, shape: {2}".format(self.__class__.__name__, x.type(), x.shape))
            print("{0}: src_states: type: {1}, shape: {2}".format(self.__class__.__name__, src_states.type(), src_states.shape))
            print("{0}: src_mask: type: {1}, shape: {2}".format(self.__class__.__name__, src_mask.type(), src_mask.shape))
            print("{0}: tgt_mask: type: {1}, shape: {2}".format(self.__class__.__name__, tgt_mask.type(), tgt_mask.shape))
        for layer in self.layers:
            x = layer(x, src_states, src_mask, tgt_mask)
        x = self.norm(x) # (batch_size, tgt_seq_len, d_model)
        if print_dims:
            print("{0}: x (output): type: {1}, shape: {2}".format(self.__class__.__name__, x.type(), x.shape))
        
        # add max pooling across sequences
        x = F.max_pool1d(x.permute(0,2,1), x.shape[1]).squeeze(-1) # (batch_size, d_model)
        return x

class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn and feed forward"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
        
    def forward(self, x, src_states, src_mask, tgt_mask):
        """norm -> self_attn -> dropout -> add -> 
        norm -> src_attn -> dropout -> add ->
        norm -> feed_forward -> dropout -> add"""
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, src_states, src_states, src_mask))
        return self.sublayer[2](x, self.feed_forward)

generator

In [ ]:
#generator
class Generator(nn.Module):
    """
    A standard linear + softmax generation step
    """
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)
        
    def forward(self, x):
        if print_dims:
            print("{0}: x: type: {1}, shape: {2}".format(self.__class__.__name__, x.type(), x.shape))
        return F.log_softmax(self.proj(x), dim=1)

Encoder Decoder

In [ ]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator, model_variant=1, graph_attention=None, concept_emb=None):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        self.model_variant = model_variant
        self.graph_attention = graph_attention
        self.concept_emb = concept_emb
        
        self.embed_size = self.src_embed[0].embedding.weight.shape[1]
        self.src_mlp = nn.Linear(2*self.embed_size, self.embed_size)

        
    def forward(self, src, tgt, src_mask, tgt_mask):
        # src: (batch_size, context_length * seq_len)
        # src_mask: (batch_size, 1, context_length * seq_len)
        # tgt: (batch_size, seq_len)
        # tgt_mask: (batch_size, 1, seq_len)
        # concept: (batch_size, (context_length + 1) * 30)

        src_embed = self.src_embed(src) # src_embed: (batch_size, context_length * seq_len, d_model)
        tgt_embed = self.tgt_embed(tgt) # tgt_embed: (batch_size, seq_len, d_model)
        if self.graph_attention is not None:
            # graph attention to compute concept representations
            src_concept_embed, tgt_concept_embed = self.graph_attention(src, src_embed, tgt, tgt_embed)

            # concatenate concept representations with utterance embeddings
            src_embed = self.src_mlp(torch.cat([src_embed, src_concept_embed], dim=-1))
            tgt_embed = self.src_mlp(torch.cat([tgt_embed, tgt_concept_embed], dim=-1))
        
        if self.model_variant == 1:
            # self-attention
            src_states = self.encoder(src_embed, src_mask)
            # cross-attention
            return self.decoder(tgt_embed, src_states, src_mask, tgt_mask)
        if self.model_variant == 2:
            seq_len = tgt.shape[1]
            context_length = src.shape[1]//seq_len
            
            # utterance-level self-attention for each src sentence
            src_states = []
            for i in range(context_length):
                s, e = i*seq_len, (i+1)*seq_len
                src_states.append(self.encoder(src_embed[:,s:e,:], src_mask[:,:,s:e]))
            src_states = torch.cat(src_states, dim=1)
            
            # context-level self-attention
            src_states = self.encoder(src_states, src_mask)
            
            # cross-attention
            return self.decoder(tgt_embed, src_states, src_mask, tgt_mask)


def make_model(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, output_size=0, dropout=0.1, KB=False, \
        model_variant=1, context_length=0, graph_attention_variant=0):
    c = copy.deepcopy
    enc_attn = MultiHeadAttention(h, d_model)
    dec_attn = MultiHeadAttention(h, d_model)
    enc_dec_attn = MultiHeadAttention(h, d_model)
    ff = PositionWiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    emb = Embeddings(d_model, src_vocab) # share src and tgt embedding
    concept_embed = None
    graph_attention = None
    if KB:
        # dynamic graph
        graph_attention = GraphAttention(vocab_size=src_vocab, embed_size=d_model, variant=graph_attention_variant, concept_embed=concept_embed)
    
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(enc_attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(dec_attn), c(enc_dec_attn), c(ff), dropout), N),
        nn.Sequential(emb, c(position)),
        nn.Sequential(emb, c(position)),
        Generator(d_model, output_size),
        model_variant=model_variant,
        graph_attention=graph_attention
    )

    return model

# Prepare batch

In [ ]:
import numpy as np
import torch

# no tgt mask for classification
def no_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size) # size: seq_len
    subsequent_mask = np.zeros(attn_shape).astype('uint8')
    # subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    # print("subsequent mask shape: ", subsequent_mask.shape) # (1, seq_len-1, seq_len-1)
    return torch.from_numpy(subsequent_mask) == 0


class ClassificationBatch:
    "Object for holding a batch of data with mask during training"
    def __init__(self, src, tgt, label, pad=0, concept=None):
        # print("src, tgt shape:", src.shape, tgt.shape)
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        self.tgt = tgt
        self.concept = concept
        self.concept_mask = None
        if concept is not None:
            self.concept_mask = (concept != pad).unsqueeze(-2)
        self.tgt_mask = (tgt != pad).unsqueeze(-2)
        self.tgt_y = label
        # self.tgt_mask = self.make_std_mask(self.tgt, pad)
        self.ntokens = torch.FloatTensor([len(src)])[0] # batch_size
        
    def to(self, device):
        self.src = self.src.to(device)
        self.src_mask = self.src_mask.to(device)
        self.tgt = self.tgt.to(device)
        self.tgt_mask = self.tgt_mask.to(device)
        if self.concept is not None:
            self.concept = self.concept.to(device)
            self.concept_mask = self.concept_mask.to(device)
        self.tgt_y = self.tgt_y.to(device)
        self.ntokens = self.ntokens.to(device)
    
    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words"
        tgt_mask = (tgt != pad).unsqueeze(-2) # (batch_size, 1, seq_len-1)
        # print("tgt_mask shape: ", tgt_mask.shape)
        tgt_mask = tgt_mask & no_mask(tgt.size(-1)).type_as(tgt_mask)
        return tgt_mask


def flatten_examples_classification(examples, vocab, k=1):
    # returns a list of ([(utter1, speaker1), (utter2, speaker2), ..., (utterk, speakerk)], label)
    import random
    classfication_examples = []
    all_speakers = list(vocab.speaker2id.values())
    empty_ex = len(examples[0][0][0])*[vocab.word2id["<pad>"]]
    for ex in examples:
        for i in range(len(ex)):
            mask = ex[i][-1]
            if mask == 1:
                if i < k:
                    context = [(empty_ex.copy(), random.choice(all_speakers)) for i in range(k-i)] # k-i
                    context += [(ex[i-j][0].copy(), ex[i-j][1]) for j in range(i, 0, -1)] # (k-i) + (i) = k
                else:
                    context = [(ex[i-j][0].copy(), ex[i-j][1]) for j in range(k, 0, -1)] # k
                if k > 0:
                    new_ex = (context + [(ex[i][0].copy(), ex[i][1])], ex[i][2])
                else:
                    new_ex = [(empty_ex.copy(), random.choice(all_speakers)), (ex[i][0].copy(), ex[i][1])], ex[i][2] # add one empty context utterance
                # if i==0:
                #     random_speaker = random.choice(all_speakers)
                #     new_ex = [(empty_ex.copy(), random_speaker), (ex[i][0].copy(), ex[i][1])], ex[i][2]
                # else:
                #     new_ex = [(ex[i-1][0].copy(), ex[i-1][1]), (ex[i][0].copy(), ex[i][1])], ex[i][2]
                classfication_examples.append(new_ex)
    return classfication_examples


def create_batches_classification(examples, batch_size, vocab, train=True):
    import random
    
    def create_one_batch(examples, vocab):
        """
            examples: a batch of examples having the same number of turns and seq_len, 
                each example is a list of ([(utter1, speaker1), ..., (utterk, speakerk), (utterA, speakerA)], label)
            return: batch_Q, batch_Q_speakers, batch_A, batch_A_speakers, batch_label
        """
        Qs = [] # context utterances
        Q_speakers = [] # context speakers
        As = [] # current utterance
        A_speakers = [] # current speaker
        labels = [] # label of current utterance

        for ex in examples:
            context = []
            context_speakers = []
            for Q, Q_speaker in ex[0][:-1]:
                context.extend(Q)
                context_speakers.append(Q_speaker)
            A, A_speaker = ex[0][-1]
            label = ex[1]
            
            Qs.append(context)
            Q_speakers.append(context_speakers)
            As.append(A)
            A_speakers.append(A_speaker)
            labels.append(label)
        
        batch = ClassificationBatch(torch.LongTensor(Qs), torch.LongTensor(As), torch.LongTensor(labels), vocab.word2id["<pad>"])
        return batch

    batch_data = []
    if train == True:
        random.shuffle(examples)
    batch_ids = list(range(0, len(examples), batch_size)) + [len(examples)]
    for s, e in zip(batch_ids[:-1], batch_ids[1:]):
        batch_examples = examples[s:e]
        one_batch = create_one_batch(batch_examples, vocab)
        batch_data.append(one_batch)
    return batch_data

# prepare loss

In [ ]:
#loss.py
import torch
import torch.nn as nn
import numpy as np
from sklearn.metrics import f1_score, mean_absolute_error, classification_report

class NoamOpt:
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
    
    def step(self):
        "Update parameter and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p["lr"] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step=None):
        if step is None:
            step = self._step
        return self.factor * (self.model_size ** (-0.5) * 
                              min(step ** (-0.5), step * self.warmup ** (-1.5)))

def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000, 
                  torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

class LabelSmoothing(nn.Module):
    "Implement label smoothing."
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(reduction="sum")
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        if print_dims:
            print("{0}: x: type: {1}, shape: {2}".format(self.__class__.__name__, x.type(), x.shape))
            print("{0}: target: type: {1}, shape: {2}".format(self.__class__.__name__, target.type(), target.shape))
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        if print_dims:
            print("{0}: true_dist: type: {1}, shape: {2}".format(self.__class__.__name__, true_dist.type(), true_dist.shape))
        return self.criterion(x, true_dist)


class SimpleLossCompute:
    "A simple loss compute and train function."
    def __init__(self, generator, criterion, dataset, emotion2id, opt=None, test=False):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        self.dataset = dataset
        self.emotion2id = emotion2id
        self.test = test
        self.outputs = []
        self.tgts = []

    @staticmethod
    def compute_score(predictions, ground, dataset, emotion2id, test=False):
        pred_y = predictions.astype(int)
        val_y = ground.astype(int)
        
        if dataset in ["EC"]:
            labels = [emotion2id["happy"], emotion2id["angry"], emotion2id["sad"]]
            score = f1_score(val_y, pred_y, average='micro', labels=labels)
            print("Micro-F1 (exclude neutral): {0}".format(score))
            if test:
                print(classification_report(val_y, pred_y, labels=labels, digits=4))
        elif dataset in ["DD"]:
            labels = [emotion2id[str(i)] for i in range(1, 7)]
            score = f1_score(val_y, pred_y, average='micro', labels=labels)
            print("Micro-F1 (exclude neutral): {0}".format(score))
            if test:
                print(classification_report(val_y, pred_y, labels=labels, digits=4))
        elif dataset in ["MELD", "IEMOCAP", "EmoryNLP"]:
            score = f1_score(val_y, pred_y, average='weighted')
            print("Weighted Macro-F1: {0}".format(score))
            if test:
                print(classification_report(val_y, pred_y, digits=4))
        else:
            score = mean_absolute_error(val_y, pred_y)
            print("MAE: {0}".format(score))
            if test:
                print(mean_absolute_error(val_y, pred_y))
        return score


    def score(self):
        score = self.compute_score(np.array(self.outputs), np.array(self.tgts), self.dataset, self.emotion2id, test=self.test)
        return score


    def clear(self):
        self.outputs = []
        self.tgts = []
        
    def __call__(self, x, y, norm):
        """
        x: (batch_size, tgt_seq_len, d_model)
        y: (batch_size, tgt_seq_len)
        norm: ()
        """
        if print_dims:
            print("{0}: x: type: {1}, shape: {2}".format(self.__class__.__name__, x.type(), x.shape))
            print("{0}: y: type: {1}, shape: {2}".format(self.__class__.__name__, y.type(), y.shape))
            print("{0}: norm: type: {1}, shape: {2}".format(self.__class__.__name__, norm.type(), norm.shape))
        x = self.generator(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)), y.contiguous().view(-1)) / norm
        self.outputs += x.contiguous().view(-1, x.size(-1)).argmax(dim=-1).tolist()
        self.tgts += y.contiguous().view(-1).tolist()
        if print_dims:
            print("{0}: loss: type: {1}, shape: {2}".format(self.__class__.__name__, loss.type(), loss.shape))
        
        if self.opt is not None:
            loss.backward()
            self.opt.step()
            if isinstance(self.opt, NoamOpt):
                self.opt.optimizer.zero_grad()
            else:
                self.opt.zero_grad()
        return loss.item() * norm

# Body program train.py

In [ ]:
logging.basicConfig(level = logging.INFO, \
                    format = '%(asctime)s  %(levelname)-5s %(message)s', \
                    datefmt =  "%Y-%m-%d-%H-%M-%S")

In [ ]:
device = torch.device(0)
test_mode = "True" # False if for test data
dataset = 'DD'
min_freq = 1
max_vocab_size = int(1e9)

In [ ]:
#this is hyperparameter
model_variant = 2
KB = "conceptnet"
KB_percentage = 1.0
graph_attention_variant = 2
GAW = -1
concentration_factor = 1
context_length = 6 
n_layers = 1
d_model = 300
d_ff = 400
h = 4

In [ ]:
if context_length == 0:
   KB = ""
embedding_size = d_model
    
if dataset == "EC":
   context_length = 2

In [ ]:
# training
epochs = 10
batch_size = 16 #semula:64
lr = 4
dropout = 0
seed = 1

In [ ]:
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [ ]:
# load examples
import pickle

logging.info("Loading data...")
train = load_pickle("/content/drive/MyDrive/Proyek Mandiri/data/DD/train.pkl".format(dataset))
val = load_pickle("/content/drive/MyDrive/Proyek Mandiri/data/DD/val.pkl".format(dataset))

2021-06-19-08-57-39  INFO  Loading data...


In [ ]:
if test_mode:
   test = load_pickle("/content/drive/MyDrive/Proyek Mandiri/data/{0}/test.pkl".format(dataset))
   train = merge_splits(train, val)
   val = test

In [ ]:
logging.info("Number of training examples: {0}".format(len(train)))
logging.info("Number of validation examples: {0}".format(len(val)))
print("Number of training examples: {0}".format(len(train)))
print("Number of validation examples: {0}".format(len(val)))

2021-06-19-08-57-43  INFO  Number of training examples: 11118
2021-06-19-08-57-43  INFO  Number of validation examples: 1000


Number of training examples: 11118
Number of validation examples: 1000


In [ ]:
for ex in train[0][:3]:
   logging.info("Examples: {0}".format(ex))
logging.info("Building vocab...")
print("Building vocab...")
vocab = Vocab(train, min_freq, max_vocab_size)
vocab_size = len(vocab.word2id)
logging.info("Vocab size: {0}".format(vocab_size))
print("Vocab size: {0}".format(vocab_size))

2021-06-19-08-58-55  INFO  Examples: (['say', ',', 'jim', ',', 'how', 'about', 'going', 'for', 'a', 'few', 'beers', 'after', 'dinner', '?'], 'Speaker A', '0', 1)
2021-06-19-08-58-55  INFO  Examples: (['you', 'know', 'that', 'is', 'tempting', 'but', 'is', 'really', 'not', 'good', 'for', 'our', 'fitness', '.'], 'Speaker B', '0', 1)
2021-06-19-08-58-55  INFO  Examples: (['what', 'do', 'you', 'mean', '?', 'it', 'will', 'help', 'us', 'to', 'relax', '.'], 'Speaker A', '0', 1)
2021-06-19-08-58-55  INFO  Building vocab...


Building vocab...


2021-06-19-08-58-56  INFO  Vocab size: 19447


Vocab size: 19447


In [ ]:
# build vocab and data    
# use pretrained word embedding
logging.info("Loading word embedding from Magnitude...")
home = os.path.expanduser("~")
if embedding_size in [50, 100, 200]:
   vectors = Magnitude("http://magnitude.plasticity.ai/glove/light/glove.twitter.27B.{0}d.magnitude".format(embedding_size))
elif embedding_size in [300]:
    # vectors = Magnitude(os.path.join(home, "WordEmbedding/GoogleNews-vectors-negative{0}.magnitude".format(embedding_size)))
    vectors = Magnitude("http://magnitude.plasticity.ai/glove/medium/glove.840B.300d.magnitude")
pretrained_word_embedding = get_vocab_embedding(vocab, vectors, embedding_size)

2021-06-19-08-59-01  INFO  Loading word embedding from Magnitude...


In [ ]:
if KB == "conceptnet":
        # Calculate edge matrix
        conceptnet = load_pickle("/content/drive/MyDrive/Proyek Mandiri/data/KB/DD.pkl".format(dataset))
        filtered_conceptnet = filter_conceptnet(conceptnet, vocab)
        filtered_conceptnet = remove_KB_duplicates(filtered_conceptnet)
        vocab_size = len(vocab.word2id)

        edge_matrix = np.zeros((vocab_size, vocab_size))
        #edge_matrix = torch.zeros([vocab_size, vocab_size], dtype=torch.float)
        for k in filtered_conceptnet:
            for c,w in filtered_conceptnet[k]:
                edge_matrix[vocab.word2id[k], vocab.word2id[c]] = w
        
        # reduce size of KB
        if KB_percentage > 0:
            logging.info("Keeping {0}% KB concepts...".format(KB_percentage*100))
            edge_matrix = edge_matrix * (np.random.random((vocab_size,vocab_size)) < KB_percentage).astype(float)
        logging.info("Edge matrix process completed")


2021-06-19-09-02-38  INFO  Keeping 100.0% KB concepts...
2021-06-19-09-02-48  INFO  Edge matrix process completed


In [ ]:
#creating tensorflow from edgematix
if KB == "conceptnet":
  edge_matrix = torch.FloatTensor(edge_matrix).to(device)
  #edge_matrix = edge_matrix.to(device) 
  edge_matrix[torch.arange(vocab_size), torch.arange(vocab_size)] = 1

In [ ]:
type(edge_matrix)

torch.Tensor

In [ ]:
edge_matrix.is_cuda

True

In [ ]:
# incorporate NRC VAD intensity

if KB == "conceptnet":
  logging.info("Loading NRC...")
  NRC = load_pickle("/content/drive/MyDrive/Proyek Mandiri/data/KB/NRC.pkl")
  affectiveness = np.zeros(vocab_size)
  #affectiveness = torch.zeros([vocab_size], dtype=torch.float)
  for w, id in vocab.word2id.items():
    VAD = get_emotion_intensity(NRC, w)
    affectiveness[id] = VAD


2021-06-19-09-03-34  INFO  Loading NRC...


In [ ]:
#create tensor for affectiveness
if KB == "conceptnet":
  affectiveness = torch.FloatTensor(affectiveness).to(device)

In [ ]:
output_size = len(vocab.emotion2id)
max_conversation_length_train = len(train[0])
max_conversation_length_val = len(val[0])
logging.info("Number of training utterances: {0}".format(vocab.num_utterances))
logging.info("Average number of training utterances per conversation: {0}".format(vocab.num_utterances/len(train)))
logging.info("Max conversation length in training set: {0}".format(max_conversation_length_train))
logging.info("Max conversation length in validation set: {0}".format(max_conversation_length_val))
logging.info("Emotion to ids: {0}".format(vocab.emotion2id))
logging.info("Emotion distribution: {0}".format(vocab.emotion_freq_dist))

2021-06-19-09-03-42  INFO  Number of training utterances: 87170
2021-06-19-09-03-42  INFO  Average number of training utterances per conversation: 7.8404389278647235
2021-06-19-09-03-42  INFO  Max conversation length in training set: 35
2021-06-19-09-03-42  INFO  Max conversation length in validation set: 31
2021-06-19-09-03-42  INFO  Emotion to ids: {'0': 0, '4': 1, '6': 2, '5': 3, '1': 4, '2': 5, '3': 6}
2021-06-19-09-03-42  INFO  Emotion distribution: Counter({'0': 72143, '4': 11182, '6': 1600, '5': 969, '1': 827, '2': 303, '3': 146})


In [ ]:
train = convert_examples_to_ids(train, vocab)
val = convert_examples_to_ids(val, vocab)


In [ ]:
train = flatten_examples_classification(train, vocab, k=context_length)
val = flatten_examples_classification(val, vocab, k=context_length)
logging.info("Batch size: {0}".format(batch_size))

2021-06-19-09-04-24  INFO  Batch size: 16


In [ ]:
 # model 
logging.info("Building model...")
model_kwargs = {
        "src_vocab": vocab_size,
        "tgt_vocab": vocab_size,
        "N": n_layers,
        "d_model": d_model,
        "d_ff": d_ff,
        "h": h,
        "output_size": output_size,
        "dropout": dropout,
        "KB": bool(KB),
        "model_variant": model_variant,
        "context_length": context_length,
        "graph_attention_variant": graph_attention_variant
}


2021-06-19-09-04-29  INFO  Building model...


In [ ]:
model = make_model(**model_kwargs)

Nilai H : 4
Nilai d_model : 300
Nilai H : 4
Nilai d_model : 300
Nilai H : 4
Nilai d_model : 300


In [ ]:
#model_iniliatization
for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    
if KB == "conceptnet":
        if GAW < 0:
            GAW = None
        model.graph_attention.init_params(GAW, edge_matrix, affectiveness, concentration_factor)

In [ ]:
logging.info("Initializing pretrained word embeddings into transformer...")
model.src_embed[0].embedding.weight.data.copy_(torch.from_numpy(pretrained_word_embedding))
model.tgt_embed[0].embedding.weight.data.copy_(torch.from_numpy(pretrained_word_embedding))
if KB != "":
     model.graph_attention.concept_embed.weight.data.copy_(torch.from_numpy(pretrained_word_embedding))
logging.info(model)
logging.info("Number of model params: {0}".format(count_parameters(model)))

2021-06-19-09-04-41  INFO  Initializing pretrained word embeddings into transformer...
2021-06-19-09-04-42  INFO  EncoderDecoder(
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiHeadAttention(
          (linears): ModuleList(
            (0): Linear(in_features=300, out_features=300, bias=True)
            (1): Linear(in_features=300, out_features=300, bias=True)
            (2): Linear(in_features=300, out_features=300, bias=True)
            (3): Linear(in_features=300, out_features=300, bias=True)
          )
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward): PositionWiseFeedForward(
          (w1): Linear(in_features=300, out_features=400, bias=True)
          (w2): Linear(in_features=400, out_features=300, bias=True)
          (dropout): Dropout(p=0, inplace=False)
        )
        (sublayer): ModuleList(
          (0): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dr

In [ ]:
model.to(device)

EncoderDecoder(
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiHeadAttention(
          (linears): ModuleList(
            (0): Linear(in_features=300, out_features=300, bias=True)
            (1): Linear(in_features=300, out_features=300, bias=True)
            (2): Linear(in_features=300, out_features=300, bias=True)
            (3): Linear(in_features=300, out_features=300, bias=True)
          )
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward): PositionWiseFeedForward(
          (w1): Linear(in_features=300, out_features=400, bias=True)
          (w2): Linear(in_features=400, out_features=300, bias=True)
          (dropout): Dropout(p=0, inplace=False)
        )
        (sublayer): ModuleList(
          (0): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0, inplace=False)
          )
          (1): SublayerConnection(
            (norm): LayerNorm()
         

In [ ]:
# weighted crossentropy loss
logging.info("Computing label weights...")
label_weight = np.array(label_distribution_transformer(val))/np.array(label_distribution_transformer(train))
label_weight = torch.tensor(label_weight/label_weight.sum()).float().to(device)*output_size
logging.info("Label weight: {0}".format(label_weight))
criterion = nn.CrossEntropyLoss(weight=label_weight, reduction="sum")
optimizer = optim.Adam(model.parameters(), lr)

2021-06-19-09-04-52  INFO  Computing label weights...
2021-06-19-09-04-52  INFO  Label weight: tensor([1.4178, 0.8802, 0.9623, 1.1732, 1.3398, 0.1425, 1.0842],
       device='cuda:0')


In [ ]:
#pake model epoch 8 dari previous training
model_save_name = 'classifier2.pt'
path = F"/content/drive/MyDrive/Proyek Mandiri/checkpint3/{model_save_name}"
model.load_state_dict(torch.load(path))

In [ ]:
# training
train_epoch_losses = []
val_epoch_losses = []
logging.info("Start training...")
for epoch in range(1, epochs + 1):
        train_batches = create_batches_classification(train, batch_size, vocab, train=True)
        val_batches = create_batches_classification(val, batch_size, vocab, train=False)
        
        train_epoch_loss = []
        val_epoch_loss = []
        model.train()
        loss_compute = SimpleLossCompute(model.generator, criterion, dataset, vocab.emotion2id, opt=optimizer, test=test_mode)

        for batch in train_batches:
            batch.to(device)
            out = model.forward(batch.src, batch.tgt, 
                                batch.src_mask, batch.tgt_mask)
            loss = loss_compute(out, batch.tgt_y, batch.ntokens)
            train_epoch_loss.append((loss/batch.ntokens).item())
        logging.info("-"*80)
        logging.info("Epoch {0}/{1}".format(epoch, epochs))
        logging.info("Training loss: {0:.4f}".format(np.mean(train_epoch_loss)))
        train_epoch_losses.append(np.mean(train_epoch_loss))
        score = loss_compute.score()
        loss_compute.clear()

        # validation
        # get src_attn
        src_attns = []
        model.eval()
        loss_compute = SimpleLossCompute(model.generator, criterion, dataset, vocab.emotion2id, opt=None, test=test_mode)
        with torch.no_grad():
            for batch in val_batches:
                batch.to(device)
                out = model.forward(batch.src, batch.tgt, 
                                    batch.src_mask, batch.tgt_mask)
                # get src attn
                src_attns.append(model.decoder.layers[0].src_attn.attn)
                loss = loss_compute(out, batch.tgt_y, batch.ntokens)
                val_epoch_loss.append((loss/batch.ntokens).item())
            logging.info("Validation loss: {0:.4f}".format(np.mean(val_epoch_loss)))
            val_epoch_losses.append(np.mean(val_epoch_loss))

        # get validation metrics
        score = loss_compute.score()
        loss_compute.clear()
        logging.info("Validation score: {0}".format(score))

        #save the model
        model_save_name = 'classifier{0}.pt'.format(epoch)
        path = F"/content/drive/MyDrive/Proyek Mandiri/checkpint4/{model_save_name}" 
        torch.save(model.state_dict(), path)
